In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image


%matplotlib inline

In [4]:
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
model = tf.keras.models.Sequential([
  
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
  
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
  
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
  
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
  
    # The fifth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

   
    
    




   
  
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1028, activation='relu'), # 512 neuron hidden layer
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# to get the summary of the model
model.summary()

# configure the model for traning by adding metrics
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics = ['accuracy'])


In [ ]:
# from keras.layers.core import Dense, Dropout, Flatten, Activation
# from keras.layers import Conv2D, MaxPooling2D
# model = tf.keras.models.Sequential([
  
#     # Note the input shape is the desired size of the image 300x300 with 3 bytes color
#     # This is the first convolution
#     tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
  
#     # The second convolution
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
  
#     # The third convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
  
#     # The fourth convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
  
#     # The fifth convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),

#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),




   
  
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1028, activation='relu'), # 512 neuron hidden layer
    
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

# # to get the summary of the model
# model.summary()

# # configure the model for traning by adding metrics
# model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics = ['accuracy'])


In [9]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.02,
        zoom_range=0.02,
        horizontal_flip=True,
        vertical_flip= True,
        rotation_range=0.01,
        )
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '../input/penyakit-orthodontal/Dentalimages/training_set',
        target_size=(300, 300),
        batch_size=50,
        class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
        '../input/penyakit-orthodontal/Dentalimages/Validation_set',
        target_size=(300, 300),
        batch_size=50,
        class_mode='binary')
history = model.fit(
         train_generator,
         epochs=100,
         validation_data=validation_generator,
      )

In [10]:
# load new unseen dataset
test_datagen = ImageDataGenerator(rescale = 1/255)

test_generator = test_datagen.flow_from_directory(
    '../input/penyakit-orthodontal/Dentalimages/test_set',
    target_size = (300, 300),
    batch_size = 10, 
    class_mode = 'binary'
)

eval_result = model.evaluate_generator(test_generator, 624)
print('loss rate at evaluation data :', eval_result[0])
print('accuracy rate at evaluation data :', eval_result[1])

In [14]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
plt.plot(loss_train, 'g', label='Training loss')
plt.plot(loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [17]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epoch = range(1,101)
plt.plot(epoch,loss_train, 'g', label='Training accuracy')
plt.plot(epoch,loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [19]:
import os
from IPython.display import Image

path = '../input/testing/nonperiodontal_1.png' 
img = image.load_img(path, target_size=(300,300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis =0)

images = np.vstack([x])
classes = model.predict(images, batch_size = 10)
print(classes[0])
if classes[0]>= 0.5:
    print('periodontal problem')
    plt.imshow(img)
else:
    print('normal')
    plt.imshow(img)